In [1]:
import sys
sys.path.append("..")
import logging
import os
from collections import OrderedDict, defaultdict

import hydra
from hydra import initialize, compose
import torch
import wandb
from fvcore.common.checkpoint import Checkpointer, PeriodicCheckpointer
from torch.cuda import amp
from torch.nn import SyncBatchNorm
from torch.nn.parallel import DistributedDataParallel
from tqdm import tqdm
import time

import detectron2.utils.comm as d2_comm
from detectron2.data.catalog import DatasetCatalog, MetadataCatalog
from detectron2.evaluation import DatasetEvaluators, inference_on_dataset
from detectron2.modeling import build_model
from detectron2.solver import build_lr_scheduler, build_optimizer
from detectron2.utils.events import CommonMetricPrinter, get_event_storage

import tridet.modeling  # pylint: disable=unused-import
import tridet.utils.comm as comm
from tridet.data import build_test_dataloader, build_train_dataloader
from tridet.data.dataset_mappers import get_dataset_mapper
from tridet.data.datasets import random_sample_dataset_dicts, register_datasets
from tridet.evaluators import get_evaluator
from tridet.modeling import build_tta_model
from tridet.utils.s3 import sync_output_dir_s3
from tridet.utils.setup import setup
from tridet.utils.train import get_inference_output_dir, print_test_results
from tridet.utils.visualization import mosaic, save_vis
from tridet.utils.wandb import flatten_dict, log_nested_dict
from tridet.visualizers import get_dataloader_visualizer, get_predictions_visualizer

# from tridet.data.datasets.kitti_3d import register_kitti_3d_datasets
from tridet.data.datasets.custom import register_kitti_3d_datasets
from tridet.data.datasets.nuscenes import register_nuscenes_datasets

from pymongo import MongoClient
from torch.utils.data import Dataset, DataLoader
from kluster import Kluster
from panoramator import Projection, Panoramator, mongo_to_shards
from detectron2.data.common import AspectRatioGroupedDataset, DatasetFromList, MapDataset
from tridet.data.augmentations import build_augmentation
from tridet.utils.tasks import TaskManager
import numpy as np
from tridet.structures.pose import Pose
from typing import List, Union
from detectron2.config import configurable
from detectron2.data import transforms as T
from tridet.evaluators.kitti_3d_evaluator import convert_3d_box_to_kitti
from tridet.utils.geometry import project_points3d
from collections import defaultdict
from copy import deepcopy
import matplotlib.pyplot as plt

In [2]:
# Panoramator structures

class PanoramaDataset(Dataset):

    def __init__(self, mongo_args, segments, keyword, projections):
        kluster = Kluster(session=MongoClient(*mongo_args))
        segments = kluster.fetch_data(
            "segments",
            {"_id": {"$in": segments}, "street_view": {"$elemMatch": {"available": True, keyword: {"$exists": False}}}}
        )
        self.kluster = mongo_args
        lines = [
            (segment["_id"], i, line["panoramas"])
            for segment in segments for i, line in enumerate(segment["street_view"])
            if "available" in line and keyword not in line
        ]
        self.panoramas = [(sid, lidx, pidx, panorama)
                          for sid, lidx, panoramas in lines for pidx, panorama in enumerate(panoramas)]
        self.projections = projections

    def __len__(self):
        return len(self.panoramas)

    def __getitem__(self, idx):
        if type(self.kluster) == tuple:
            self.kluster = Kluster(session=MongoClient(*self.kluster))
        segment_id, line_idx, panorama_idx, panorama_id = self.panoramas[idx]
        panorama = main_kluster.kluster["street_view"].find_one({"_id": panorama_id})
        shards = mongo_to_shards(panorama["panorama"])
        panoramator = Panoramator(shards=shards, atomic_resolution=panorama["resolution"][0]//16)
        panoramator.build_state()
        projections = [(projection_meta, panoramator.get_projection(projection_meta))
                       for projection_meta in self.projections]
        return segment_id, line_idx, panorama_id, projections

    
def inference(kluster, predictor, data_loader, keyword):
    current_line = None
    line_count = 0

    for i, (segment_id, line_idx, panorama_id, projections) in enumerate(data_loader):
        itime = time.time()

        if current_line is not None and current_line != (segment_id, line_idx):
            sid, lidx = current_line
            # kluster.kluster["segments"].update_one({"_id": sid}, {"$set": {f"street_view.{lidx}.{keyword}": True}})
            line_count += 1
            print(f"Finished line {line_count}! (Segment:{sid};Index:{lidx})")
        current_line = (segment_id, line_idx)

        result = []
        for projection_meta, projection in projections:
            predictions = predictor(projection)
            result.append({"projection": projection_meta.get_dict(), **predictions})
        # kluster.kluster["street_view"].update_one({"_id": panorama_id}, {"$set": {keyword: result}})

        print(f"Predicted panorama {i+1}/{len(data_loader)} (Time elapsed: {time.time()-itime:.2f}s) ({panorama_id})")

In [144]:
# DD3D structures

class ParkinkDatasetMapper:
    
    @configurable
    def __init__(self, is_train: bool, task_manager, augmentations: List[Union[T.Augmentation, T.Transform]], 
                 image_format: str, intrinsics: list, extrinsics: dict):
        self.is_train = is_train
        self.task_manager = task_manager
        self.augmentations = T.AugmentationList(augmentations)
        print("Augmentations used: " + str(augmentations))
        self.image_format = image_format
        self.intrinsics = intrinsics
        self.extrinsics = extrinsics

    @classmethod
    def from_config(cls, cfg, is_train, intrinsics, extrinsics):
        augs = build_augmentation(cfg, is_train)
        tm = TaskManager(cfg)
        return {"is_train": is_train, "task_manager": tm, "augmentations": augs, "image_format": cfg.INPUT.FORMAT, 
               "intrinsics": intrinsics, "extrinsics": extrinsics}

    def __call__(self, parkink_data):
        """
        Args:
            dataset_dict (dict): Metadata of one image, in Detectron2 Dataset format.
        Returns:
            dict: a format that builtin models in detectron2 accept
        """
        segment_id, line_idx, panorama_id, projections = parkink_data
        
        kitti_projections = []
        for projection_meta, image in projections:
            kitti = {"width": image.shape[1], "height": image.shape[0],
                     "intrinsics": self.intrinsics, "extrinsics": self.extrinsics}
            
            if type(image) == torch.Tensor:  # When using a DataLoader, Tensors instead of arrays will be given
                image = image.numpy()
            image = image[:, :, ::-1]  # VERY IMPORTANT! CONVERT IMAGE FROM RGB (PIL format) TO BGR (model format)
            aug_input = T.AugInput(image)
            transforms = self.augmentations(aug_input)
            image = aug_input.image
            kitti["image"] = torch.as_tensor(np.ascontiguousarray(image.transpose(2, 0, 1)))
            
            intrinsics = np.reshape(kitti["intrinsics"], (3, 3)).astype(np.float32)
            intrinsics = transforms.apply_intrinsics(intrinsics)
            kitti["intrinsics"] = torch.as_tensor(intrinsics)
            kitti["inv_intrinsics"] = torch.as_tensor(np.linalg.inv(intrinsics))
            
            extrinsics = Pose(wxyz=np.float32(kitti["extrinsics"]["wxyz"]),
                              tvec=np.float32(kitti["extrinsics"]["tvec"]))
            kitti["extrinsics"] = extrinsics
            
            kitti_projections.append((projection_meta, kitti))
            
        return segment_id, line_idx, panorama_id, kitti_projections


def meter_to_angle(x, y, z):
    # Convert meters coordinates to horizontal and vertical angles.
    # We negate the vertical and so that up is positive and down is negative.
    return np.array([np.arctan2(x, z), -np.arctan2(y, z)]) / np.pi * 180

def process_scene(model, input_dict, plot=False, log=False):
    CLASS_MAPPER = ("Car", "Pedestrian", "Cyclist", "Van", "Truck")
    THRESHOLD = 0.5
    with torch.no_grad():
        raw_output = model([input_dict])[0]
    instances = raw_output["instances"].get_fields()
    
    # We discard 
    # - instances["scores"]: scores_3d gives a more informed score, taking into account the 3d box
    # - instances["locations"]: this is ~= object center, but the center given by 3d box is more accurate
    # - instances["pred_boxes"]: 2d boxes, a priori useless if we have 3d ones (is this an intermediate step?)
    # - instances["fpn_levels"]: This is related to at which level of the net the object is detected... useless
    zipped = zip(instances["scores_3d"], instances["pred_classes"], instances["pred_boxes3d"])
    subd = {"pixels": [], "meters": [], "degrees": []}
    prediction = {"score": [], "kitti_class": [], "size": [], "orientation": [], "center": deepcopy(subd),
                  "front_upper_left": deepcopy(subd), "front_upper_right": deepcopy(subd),
                  "front_lower_right": deepcopy(subd), "front_lower_left": deepcopy(subd),
                  "back_upper_left": deepcopy(subd), "back_upper_right": deepcopy(subd),
                  "back_lower_right": deepcopy(subd), "back_lower_left": deepcopy(subd)}
    for score_3d, kitti_class, box_3d in zipped:
        if score_3d < THRESHOLD:  # If the model is not confident enough, we skip the prediction
            continue
        prediction["score"].append(round(score_3d.item(), 3))
        if kitti_class not in (0, 3, 4):  # If the detected object is not a car, van or truck, we skip it
            continue
        kitti_class = CLASS_MAPPER[kitti_class]
        prediction["kitti_class"].append(kitti_class)
        
        center_pix = box_3d.proj_ctr[0].cpu().numpy()  # width (x), height (y)
        center_met = box_3d.tvec[0].cpu().numpy()  # horizontal (left->right), vertical (up->down), depth (back->front)
        center_ang = meter_to_angle(*center_met)  # horizontal (left->right, degrees), vertical (down->up, degrees)
        prediction["center"]["pixels"].append([round(e, 1) for e in center_pix.tolist()])
        prediction["center"]["meters"].append([round(e, 2) for e in center_met.tolist()])
        prediction["center"]["degrees"].append([round(e, 2) for e in center_ang.tolist()])
        
        size = box_3d.size[0].cpu().numpy()  # width, length, height (meters)
        prediction["size"].append([round(e, 2) for e in size.tolist()])
        
        corners_met = box_3d.corners[0].cpu().numpy()
        corners_ang = np.array([meter_to_angle(*corner) for corner in corners_met])
        corners_pix = project_points3d(corners_met, input_dict["intrinsics"].numpy())
        corners_pix = [pix * (-1 if met[2] < 0 else 1) for met, pix in zip(corners_met, corners_pix)]
        keys = ["front_upper_left", "front_upper_right", "front_lower_right", "front_lower_left", 
                "back_upper_left", "back_upper_right", "back_lower_right", "back_lower_left"]
        for key, pix, met, ang in zip(keys, corners_pix, corners_met, corners_ang):
            for e in pix.tolist():
                print(e)
            prediction[key]["pixels"].append([round(e, 1) for e in pix.tolist()])
            prediction[key]["meters"].append([round(e, 2) for e in met.tolist()])
            prediction[key]["degrees"].append([round(e, 2) for e in ang.tolist()])
            
        
        w, l, h, x, y, z, roty, alpha = convert_3d_box_to_kitti(box_3d)
        orientation = (roty / np.pi * 180) + 90  # This orientation represents the horizontal car deviation from the street forward direction
        prediction["orientation"].append(round(orientation, 2))
        
        if log:
            print(f"Confidence: {score_3d}")
            print(f"Class: {kitti_class}")
            print(f"Center (pixels): {center_pix}")
            print(f"Center (meters): {center_met}")
            print(f"Center (degrees): {center_ang}")
            print(f"Size (meters): {size}")
            print(f"Corners (pixels): {corners_pix}")
            print(f"Corners (meters): {corners_met}")
            print(f"Corners (degrees): {corners_ang}")
            print(f"Car Orientation (degrees): {orientation}")
        
        if plot:
            for a, b, c, d in [(0, 1, 2, 3), (4, 5, 6, 7), (0, 4, 7, 3), (1, 5, 6, 2), (0, 2, 1, 3)]:
                coord = [corners_pix[a], corners_pix[b], corners_pix[c], corners_pix[d], corners_pix[a]] 
                xs, ys = zip(*coord)
                plt.plot(xs, ys, color='r')
    
    if plot:
        img = input_dict["image"].cpu().numpy().transpose(1, 2, 0)[:, :, ::-1]
        plt.rcParams['figure.figsize'] = [10, 5]
        plt.imshow(img)
        plt.show()
    
    from pprint import pprint
    pprint(prediction)
    
    return prediction

In [145]:
CFG_PATH = "../configs/"
CFG_NAME = "kitti99_defaults"
CHECKPOINT = "../models/kitti_v99.pth"
INTRINSICS = [612.6, 0.0, 640.0, 0.0, 612.6, 128.0, 0.0, 0.0, 1.0]
EXTRINSICS = {"wxyz": [1.0, 0.0, 0.0, 0.0], "tvec": [0.0, 0.0, 0.0]}

In [146]:
MONGO_SESSION_ARGS = ("localhost", 27017)
PREDICTION_KEYWORD = "kitti_cars"
TIMEOUT = 180
PROJECTIONS = [Projection(center_horizontal=0, center_vertical=0, fov_horizontal=92.5, fov_vertical=45.36,
                          full_resolution_x=1280, full_resolution_y=512,
                          offset_x=0, offset_y=512-384, resolution_x=1280, resolution_y=384),
               Projection(center_horizontal=180, center_vertical=0, fov_horizontal=92.5, fov_vertical=45.36,
                          full_resolution_x=1280, full_resolution_y=512,
                          offset_x=0, offset_y=512-384, resolution_x=1280, resolution_y=384)]
MIN_LAT, MAX_LAT = 41.35, 41.5
MIN_LON, MAX_LON = 2.1, 2.3
PLOT = False
LOG = False

In [147]:
# StreetView initializations
main_kluster = Kluster(session=MongoClient(*MONGO_SESSION_ARGS))
bounding_polygon = [(MIN_LAT, MIN_LON), (MIN_LAT, MAX_LON), (MAX_LAT, MAX_LON),
                    (MAX_LAT, MIN_LON), (MIN_LAT, MIN_LON)]
bounding_polygon = {"type": "Polygon", "coordinates": [[[lon, lat] for lat, lon in bounding_polygon]]}

In [148]:
# DD3D initializations
with initialize(config_path=CFG_PATH):
    cfg = compose(config_name=CFG_NAME)
setup(cfg)
dd3d_model = build_model(cfg).eval()
Checkpointer(dd3d_model).load(CHECKPOINT)
dd3d_predictor = lambda image: process_scene(dd3d_model, image, plot=PLOT, log=LOG)

In [149]:
# Load segment_ids of interest
ways = main_kluster.fetch_data("ways", {"path": {"$geoIntersects": {"$geometry": bounding_polygon}}})
segment_ids = [seg_id for way in ways for seg_id in way["segments"].values()]

In [150]:
# Dataset and Loader
dataset = PanoramaDataset(MONGO_SESSION_ARGS, segment_ids, PREDICTION_KEYWORD, PROJECTIONS)
mapper = ParkinkDatasetMapper(cfg, is_train=False, intrinsics=INTRINSICS, extrinsics=EXTRINSICS)
dataset = MapDataset(dataset, mapper)
loader = DataLoader(dataset, batch_size=None, num_workers=0)

Augmentations used: [ResizeShortestEdge(short_edge_length=(384, 384), max_size=100000, sample_style='choice')]


In [151]:
inference(main_kluster, dd3d_predictor, loader, PREDICTION_KEYWORD)

500.2314758300781
162.1988525390625
555.1652221679688
161.587890625
555.3487548828125
213.6372833251953
500.158935546875
214.33531188964844
468.1852111816406
162.294677734375
536.7933349609375
161.53216552734375
536.9156494140625
226.5517578125
467.9075927734375
227.4501953125
452.7109680175781
172.8733367919922
526.9990234375
171.4889373779297
527.440673828125
237.8920440673828
452.6490173339844
239.91453552246094
403.1257019042969
174.8516082763672
504.0838928222656
172.95693969726562
504.47174072265625
262.8686218261719
402.5812072753906
265.9371032714844
224.45025634765625
224.0476531982422
157.77444458007812
211.43667602539062
163.23402404785156
366.119873046875
230.94754028320312
410.7569580078125
559.597412109375
180.86380004882812
477.1828308105469
177.4203338623047
480.5597229003906
280.0821533203125
563.3367309570312
296.71429443359375
-225.4814910888672
233.02847290039062
165.9980926513672
195.59747314453125
177.17469787597656
390.5551452636719
-212.88551330566406
511.135101

465.9310302734375
145.5759735107422
506.7937927246094
145.53536987304688
506.5909118652344
184.25460815429688
465.6437683105469
185.88906860351562
505.2188720703125
145.55430603027344
550.155029296875
145.50888061523438
550.0115966796875
189.77841186523438
504.9725341796875
191.91976928710938
399.5535888671875
188.65231323242188
521.7551879882812
184.00228881835938
524.3809814453125
297.690185546875
400.5414733886719
303.8241271972656
251.3011016845703
199.15054321289062
463.69427490234375
190.96728515625
466.8794860839844
387.7420349121094
249.48768615722656
400.41119384765625
629.228515625
140.11769104003906
635.2172241210938
139.28564453125
635.0671997070312
167.88499450683594
629.0742797851562
167.2248992919922
549.1675415039062
139.32852172851562
550.7958374023438
138.42959594726562
550.4579467773438
167.7952423095703
548.8444213867188
167.1231689453125
225.52386474609375
139.84959411621094
243.26263427734375
139.42543029785156
243.3595733642578
165.09445190429688
225.622299194335

1029.30859375
141.31918334960938
970.6621704101562
140.58384704589844
970.5025634765625
194.504150390625
1029.19775390625
199.16848754882812
1074.7655029296875
141.35958862304688
1019.6173706054688
140.70022583007812
1019.5167236328125
188.86634826660156
1074.708740234375
192.63671875
178.9481964111328
156.02684020996094
208.98927307128906
155.3280029296875
209.02304077148438
188.8975372314453
178.93492126464844
191.7854461669922
268.9104309082031
154.16505432128906
293.62921142578125
153.57582092285156
293.7944030761719
187.3807830810547
269.045654296875
190.19091796875
837.897216796875
141.6785430908203
864.1851196289062
141.7801513671875
864.2738647460938
178.66650390625
837.9484252929688
176.45973205566406
761.0980224609375
141.2118682861328
784.0206909179688
141.28961181640625
784.0035400390625
180.35394287109375
761.0521240234375
177.92303466796875
351.01422119140625
165.1915740966797
430.2689514160156
163.2705535888672
430.69561767578125
231.92332458496094
351.0336608886719
238.

998.8359985351562
151.39602661132812
1050.21044921875
152.2913055419922
1050.5224609375
214.1040802001953
998.9907836914062
207.19976806640625
870.9808349609375
149.81349182128906
910.5547485351562
150.57595825195312
910.4812622070312
215.5515899658203
870.8125610351562
208.18228149414062
454.9820251464844
165.79266357421875
481.4548645019531
164.9351806640625
481.3166809082031
233.46437072753906
454.65252685546875
244.07252502441406
657.0643310546875
163.61013793945312
658.5398559570312
163.01487731933594
659.4190673828125
232.4681396484375
658.0608520507812
243.0980682373047
-27.160743713378906
159.80043029785156
38.297210693359375
158.4775848388672
38.159584045410156
210.8197479248047
-27.499818801879883
217.39108276367188
92.7760009765625
156.4211883544922
148.26361083984375
155.3881072998047
148.4149169921875
208.61126708984375
92.78076171875
215.08018493652344
679.0178833007812
165.5172119140625
679.9484252929688
165.0846405029297
678.3535766601562
239.9834442138672
677.169128417

Finished line 1! (Segment:01113fefa81242f887a1045c;Index:0)
851.1563720703125
159.92686462402344
727.0347900390625
158.2698974609375
726.363525390625
280.035400390625
851.8814086914062
282.3376159667969
787.7379760742188
160.8811492919922
703.8281860351562
159.75482177734375
703.1990356445312
241.99925231933594
787.74462890625
243.41949462890625
474.5481262207031
187.28465270996094
207.71109008789062
197.982421875
200.55833435058594
482.3835144042969
478.7469787597656
464.3809814453125
537.8056030273438
186.923828125
399.5480041503906
192.49356079101562
400.1008605957031
337.5780334472656
541.34423828125
330.08056640625
511.4526062011719
165.20867919921875
574.1884765625
164.52183532714844
574.45849609375
223.45126342773438
511.34326171875
224.68797302246094
491.34942626953125
164.6643829345703
571.3515625
163.79736328125
571.6736450195312
238.8437957763672
491.0535888671875
240.60482788085938
467.643798828125
174.98800659179688
554.6061401367188
173.8282928466797
554.8795776367188
252

...

520.4466552734375
178.60226440429688
386.9509582519531
181.86268615722656
385.8454284667969
314.5623779296875
521.6127319335938
309.01068115234375
543.3783569335938
178.42617797851562
452.93212890625
180.6395721435547
452.950439453125
270.2999572753906
544.4336547851562
267.0345153808594
776.9429321289062
159.08401489257812
695.9572143554688
158.7238311767578
695.9832763671875
241.71722412109375
777.5297241210938
241.6907501220703
739.9566650390625
159.16506958007812
677.7860107421875
158.8894500732422
677.709228515625
222.36305236816406
740.2098388671875
222.41226196289062
670.2644653320312
116.97347259521484
637.1246948242188
117.11994171142578
637.2595825195312
145.92628479003906
670.37646484375
145.68923950195312
663.255859375
117.21992492675781
633.6251220703125
117.35149383544922
633.747802734375
143.09214782714844
663.3601684570312
142.88821411132812
348.1508483886719
267.2278747558594
-33.94140625
278.3682556152344
-36.458274841308594
626.7831420898438
351.84649658203125
610.24

591.2289428710938
103.25714874267578
550.1935424804688
102.87020874023438
550.0360717773438
140.667724609375
590.9935302734375
140.85459899902344
591.2330932617188
103.87275695800781
555.1539306640625
103.5354232788086
555.007080078125
136.77439880371094
591.0259399414062
136.9569091796875
458.2030334472656
194.83489990234375
148.71536254882812
207.56944274902344
140.1549835205078
502.07232666015625
462.49017333984375
485.0780334472656
548.0494384765625
188.1328887939453
404.71405029296875
194.0135040283203
405.6744079589844
327.6544494628906
551.705322265625
320.8869323730469
710.17919921875
113.27495574951172
665.3447265625
113.14492797851562
665.1845092773438
153.0989532470703
709.944580078125
153.1160430908203
696.3236694335938
113.11697387695312
658.0973510742188
113.00582885742188
657.97119140625
147.0498504638672
696.1433715820312
147.07896423339844
671.1259155273438
99.32071685791016
641.2666625976562
99.42623138427734
641.372314453125
127.72330474853516
671.1769409179688
127.5

512.2621459960938
105.74105834960938
560.480224609375
107.7115707397461
558.95849609375
153.1678466796875
510.7462463378906
152.55117797851562
531.55224609375
104.87657928466797
586.918212890625
107.19471740722656
585.1190795898438
159.99200439453125
529.7591552734375
159.5091552734375
687.4043579101562
106.56610870361328
646.9998168945312
106.33509063720703
646.7617797851562
144.3973846435547
687.0740356445312
144.47390747070312
676.1218872070312
106.34381866455078
641.0631713867188
106.14280700683594
640.8687744140625
139.14219665527344
675.85791015625
139.22702026367188
757.075927734375
123.2816162109375
683.9285888671875
123.40837860107422
683.98095703125
192.6277313232422
757.020751953125
192.4404296875
731.5181884765625
122.59843444824219
673.7384033203125
122.69805908203125
673.7916259765625
177.37608337402344
731.5042724609375
177.23867797851562
660.2368774414062
94.38418579101562
626.8713989257812
94.3954086303711
626.9517211914062
131.59596252441406
660.2233276367188
131.3846

714.7378540039062
152.79925537109375
649.797119140625
152.51380920410156
649.5502319335938
216.96205139160156
714.7564697265625
216.80645751953125
692.648193359375
152.58326721191406
640.2008666992188
152.35206604003906
639.970458984375
204.2383575439453
692.5908813476562
204.1833038330078
941.5704345703125
180.31785583496094
744.2037963867188
176.02420043945312
743.5316162109375
387.5386047363281
947.5997314453125
388.1489562988281
810.714111328125
183.69667053222656
689.6027221679688
181.12921142578125
688.0728759765625
307.7928466796875
811.6749877929688
309.03076171875
505.8367614746094
160.29241943359375
385.0126037597656
162.0426025390625
384.20062255859375
281.3331604003906
506.23016357421875
277.4587097167969
529.1883544921875
160.34835815429688
445.22216796875
161.56951904296875
445.0777282714844
244.50096130371094
529.624267578125
242.2474822998047
670.24072265625
134.5347137451172
619.8209228515625
134.47230529785156
619.7582397460938
190.4957275390625
670.2610473632812
190.

KeyboardInterrupt: 

Mapper should match keys `instrinsics`, `width`, `height`, `extrinsics` from 
{'intrinsics': [612.6, 0.0, 640.0, 0.0, 612.6, 128.0, 0.0, 0.0, 1.0],
 'file_name': '/workspace/dd3d/datasets/KITTI3D/inference/image_2/a_back.jpg',
 'width': 1280,
 'height': 384,
 'image_id': 'a_back_camera_2',
 'sample_id': 'a_back',
 'extrinsics': {'wxyz': [1.0, 0.0, 0.0, 0.0], 'tvec': [0.0, 0.0, 0.0]}}
for each projection element in the list.

Additionally, it should follow the default mapper logic to obtain the mapped dataset. i.e.
{'intrinsics': tensor([[612.6000,   0.0000, 640.0000],
        [  0.0000, 612.6000, 128.0000],
        [  0.0000,   0.0000,   1.0000]]), 'file_name': '/workspace/dd3d/datasets/KITTI3D/inference/image_2/a_back.jpg', 'width': 1280, 'height': 384, 'image_id': 'a_back_camera_2', 'sample_id': 'a_back', 'extrinsics': wxyz: +1.000 +0.000i +0.000j +0.000k, tvec: ([0.00 0.00 0.00]), 'image': tensor([[[100, 127, 144,  ...,  62,  80,  95],
         [106, 120, 126,  ...,  67,  86,  99],
         [ 72,  82,  91,  ...,  69,  88, 100],
         ...,
         [254, 254, 254,  ...,  54,  67,  67],
         [254, 254, 254,  ...,  37,  53,  60],
         [254, 254, 254,  ...,  14,  32,  42]],

        [[120, 147, 164,  ...,  71,  89, 104],
         [127, 141, 147,  ...,  76,  95, 108],
         [ 94, 104, 113,  ...,  78,  97, 109],
         ...,
         [254, 254, 254,  ...,  75,  86,  86],
         [254, 254, 254,  ...,  58,  71,  78],
         [254, 254, 254,  ...,  35,  50,  60]],

        [[121, 148, 165,  ...,  75,  93, 108],
         [128, 142, 148,  ...,  80,  99, 112],
         [ 92, 102, 111,  ...,  82, 101, 113],
         ...,
         [254, 254, 254,  ...,  72,  83,  83],
         [254, 254, 254,  ...,  56,  70,  77],
         [254, 254, 254,  ...,  33,  49,  59]]], dtype=torch.uint8), 'inv_intrinsics': tensor([[ 0.0016,  0.0000, -1.0447],
        [ 0.0000,  0.0016, -0.2089],
        [ 0.0000,  0.0000,  1.0000]])}